In [2]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local[*]")\
        .config("spark.driver.memory","5g")\
        .appName('Trail_Transformations')\
        .getOrCreate()

In [3]:
file_path = "./data/customer_data.csv"
df = spark.read.csv(file_path, header = True,inferSchema=True)

In [4]:
df = df.withColumn("Credit_Card_Number", df["Credit_Card_Number"].cast("double"))
df.printSchema()
df.show()

root
 |-- ID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Email: string (nullable = true)
 |-- Credit_Card_Number: double (nullable = true)
 |-- Credit_Card_Type: string (nullable = true)

+---+----------------+--------------------+--------------------+--------------------+
| ID|            Name|               Email|  Credit_Card_Number|    Credit_Card_Type|
+---+----------------+--------------------+--------------------+--------------------+
|  1|      Rodi Lotze|  rlotze0@netlog.com|                 NaN|      china-unionpay|
|  2| Vachel Waistell|vwaistell1@answer...|                 NaN|diners-club-inter...|
|  3|  Mikey Buttrick| mbuttrick2@narod.ru|      5.893663168E17|             maestro|
|  4|            None|mking3@networkadv...|3.539133178799873E15|                 jcb|
|  5| Beverie Ternott|bternott4@rakuten...|3.528122067307396E15|                 jcb|
|  6|Adan Spain-Gower|aspaingower5@diig...|3.552301123264739E15|                 jcb|
|  7|  Julius 

In [6]:
df.count()
df1=df.dropna()
df1.count()

789

In [13]:
unique_card = df1.select("Credit_Card_Type").distinct().rdd.map(lambda x: x[0]).collect()

['jcb', 'visa-electron', 'maestro', 'diners-club-international', 'solo', 'americanexpress', 'diners-club-enroute', 'china-unionpay', 'bankcard', 'switch', 'mastercard', 'instapayment', 'visa', 'diners-club-carte-blanche', 'laser', 'diners-club-us-ca']
<class 'list'>
16


24/04/17 16:01:15 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1751032 ms exceeds timeout 120000 ms
24/04/17 16:01:15 WARN SparkContext: Killing executors is not supported by current scheduler.
24/04/17 16:01:16 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$

# Show all the columns and their data types.

In [21]:
df.dtypes
#df.columns

[('id', 'int'),
 ('updated', 'date'),
 ('confirmed', 'int'),
 ('confirmed_change', 'int'),
 ('deaths', 'int'),
 ('deaths_change', 'smallint'),
 ('recovered', 'int'),
 ('recovered_change', 'int'),
 ('latitude', 'double'),
 ('longitude', 'double'),
 ('iso2', 'string'),
 ('iso3', 'string'),
 ('country_region', 'string'),
 ('admin_region_1', 'string'),
 ('iso_subdivision', 'string'),
 ('admin_region_2', 'string'),
 ('load_time', 'timestamp')]

In [15]:
from pyspark.sql import Row
import pyspark.sql.functions as F
eDF = spark.createDataFrame([Row(a=1, intlist=[1,2,3], mapfield={"a":"b"})])
eDF.show()
print(F.explode(eDF.intlist).alias("anInt"))
eDF.select(F.explode(eDF.mapfield).alias("a","b")).show()

+---+---------+--------+
|  a|  intlist|mapfield|
+---+---------+--------+
|  1|[1, 2, 3]|{a -> b}|
+---+---------+--------+

Column<'explode(intlist) AS anInt'>
+---+---+
|  a|  b|
+---+---+
|  a|  b|
+---+---+



In [24]:
import pyspark
from pyspark.sql import SparkSession

arrayData = [
        ('James',['Java','Scala'],{'hair':'black','eye':'brown'}),
        ('Michael',['Spark','Java',None],{'hair':'brown','eye':None}),
        ('Robert',['CSharp',''],{'hair':'red','eye':''}),
        ('Washington',None,None),
        ('Jefferson',['1','2'],{})]
df = spark.createDataFrame(data=arrayData, schema = ['name','knownLanguages','properties'])

from pyspark.sql.functions import explode
df2 = df.select(df.name,explode(df.knownLanguages))
print(explode(df.knownLanguages))
print(df.name)
print(df.knownLanguages)
df2.printSchema()
df2.show()

Column<'explode(knownLanguages)'>
Column<'name'>
Column<'knownLanguages'>
root
 |-- name: string (nullable = true)
 |-- col: string (nullable = true)

+---------+------+
|     name|   col|
+---------+------+
|    James|  Java|
|    James| Scala|
|  Michael| Spark|
|  Michael|  Java|
|  Michael|  NULL|
|   Robert|CSharp|
|   Robert|      |
|Jefferson|     1|
|Jefferson|     2|
+---------+------+



In [53]:
columns = ["Seqno","Name"]
data = [("1", "john jones"),
    ("2", "tracey smith"),
    ("3", "amy sanders"),
    ("4","hello")]

# Create DataFrame
df = spark.createDataFrame(data=data,schema=columns)
df.show()
# foreach() Example
def f(df):
    print(df.Seqno,df.Name)
df.foreach(f)
accum=spark.sparkContext.accumulator(0)
df.foreach(lambda x:accum.add(int(x.Seqno)))
print(accum.value) #Accessed by driver

+-----+------------+
|Seqno|        Name|
+-----+------------+
|    1|  john jones|
|    2|tracey smith|
|    3| amy sanders|
|    4|       hello|
+-----+------------+



1 john jones
2 tracey smith
4 hello
3 amy sanders


10


24/04/01 20:18:43 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 2028261 ms exceeds timeout 120000 ms
24/04/01 20:18:43 WARN SparkContext: Killing executors is not supported by current scheduler.
24/04/01 20:18:43 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at 

In [18]:
spark.stop()